# visalise Session as Scroll using Ploty `Session.plotscroll()`

```bash
jupyter nbconvert "D:\OneDrive - Nexus365\Private_Dropbox\Projects\trialexp\notebooks\noncanonical\nb20221017_104300_session_plotscroll.ipynb" --to="python" --output-dir="D:\OneDrive - Nexus365\Private_Dropbox\Projects\trialexp\notebooks\noncanonical" --output="nb20221017_104300_session_plotscroll"
```

## Requirements

Plotly `conda install plotly`


    fftw-3.3.9                 |       h2bbff1b_1         672 KB
    icc_rt-2022.1.0            |       h6049295_2         6.5 MB
    numpy-1.23.1               |   py38h7a0a035_0          10 KB
    numpy-base-1.23.1          |   py38hca35cd5_0         5.0 MB
    plotly-5.9.0               |   py38haa95532_0         4.1 MB
    scikit-learn-1.1.2         |   py38hd77b12b_0         5.5 MB
    scipy-1.9.1                |   py38he11b74f_0        15.7 MB
    setuptools-63.4.1          |   py38haa95532_0         1.0 MB
    tenacity-8.0.1             |   py38haa95532_1          34 KB

### Imports

In [1]:
# allow for automatic reloading of classes and function when updating the code
%load_ext autoreload
%autoreload 2 

# Import Session and Experiment class with helper functions
from trialexp.process.data_import import *

### Variables

In [2]:
import pandas as pd

trial_window = [-2000, 6000] # in ms

# time limit around trigger to perform an event
# determine successful trials
timelim = [0, 2000] # in ms

# Digital channel nb of the pyphotometry device
# on which rsync signal is sent (from pycontrol device)
rsync_chan = 2

basefolder, _ = os.path.split(os.path.split(os.getcwd())[0])

# These must be absolute paths
# use this to use within package tasks files (in params)
tasksfile = os.path.join(basefolder,'params\\tasks_params.csv')
# use this to put a local full path
#tasksfile = -r'C:/.../tasks_params.csv' 

# photometry_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\test_folder\photometry'
photometry_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\kms_pyphotometry'
video_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\videos'


### Tasks
- A tasks definition file (.csv) contains all the information to perform the extractions of behaviorally relevant information from **PyControl** files, for each **task** file. It includes what are the **triggers** of different trial types, what **events** to extract (with time data), and what are events or printed lines that could be relevant to determine the **conditions** (e.g: free reward, optogenetic stimulation type, etc.)
- To analyze a new task you need to append task characteristics like **task** filename, **triggers**, **events** and **conditions**

In [3]:
tasks = pd.read_csv(tasksfile, usecols = [1,2,3,4], index_col = False)
tasks

,task,triggers,events,conditions
0,lick_go_nogo,go; nogo,bar; bar_off; spout,free_reward
1,lick_go_nogo_unconditionned,go; nogo,bar; bar_off; spout,free_reward
2,lick_go_nogo_unconditionned_opto,go; nogo,bar; bar_off; spout,free_reward
3,reaching_go_nogo,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
4,reaching_go_nogo_jc,CS_plus; CS_minus,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
5,reaching_go_nogo_opto_continuous,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset; s_go_sham; s_go_...
6,reaching_go_nogo_opto_sinusoid,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset_2; s_nogo_cs_onse...
7,reaching_go_nogo_opto_sinusoid_spout,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset_2; s_nogo_cs_onse...
8,reaching_go_nogo_reversal,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
9,reaching_go_nogo_reversal_incentive,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...


### Optional

Transfer Files from hierarchical folders by tasks to flat folders, for photometry and behaviour files

2m 13.9s

If we obtain list of files in source and dest at first and then only perform comparison on them,
This should be much faster

In [4]:
photo_root_dir = 'T:\\Data\\head-fixed\\pyphotometry\\data'
pycontrol_root_dir = 'T:\\Data\\head-fixed\\pycontrol'

root_folders = [photo_root_dir, pycontrol_root_dir]
horizontal_folder_pycontrol = 'T:\\Data\\head-fixed\\test_folder\\pycontrol'
horizontal_folder_photometry = 'T:\\Data\\head-fixed\\test_folder\\photometry'

copy_files_to_horizontal_folders(root_folders, horizontal_folder_pycontrol, horizontal_folder_photometry)

KeyboardInterrupt: 

### Create an experiment object

This will include all the pycontrol files present in the folder_path directory (do not include subdirectories)

In [5]:
# Folder of a full experimental batch, all animals included

# Enter absolute path like this
# pycontrol_files_path = r'T:\Data\head-fixed\test_folder\pycontrol'

# or this if you want to use data from the sample_data folder within the package
pycontrol_files_path = os.path.join(basefolder,'sample_data/pycontrol')
pycontrol_files_path = r'T:\Data\head-fixed\kms_pycontrol'

# Load all raw text sessions in the indicated folder or a sessions.pkl file
# if already existing in folder_path
exp_cohort = Experiment(pycontrol_files_path)

# Only use if the Experiment cohort as been processed by trials before
# TODO: assess whether this can be removed or not
exp_cohort.by_trial = True

Saved sessions loaded from: sessions.pkl


### Perform extraction of behavioural information by trial

5m55.4s

In [6]:
# Process the whole experimental folder by trials
exp_cohort.process_exp_by_trial(trial_window, timelim, tasksfile, blank_spurious_event='spout', blank_timelim=[0, 65])

# Save the file as sessions.pkl in folder_path
# exp_cohort.save() # Do I need to save this???


No event  'spout'  found:  45 2022-04-27 15:57:48 pavlovian_nobar_nodelay
No event  'spout'  found:  42 2022-04-28 12:53:21 pavlovian_nobar_nodelay
No event  'spout'  found:  45 2022-05-06 12:27:18 pavlovian_nobar_nodelay
reaching_go_spout_cued_uncued 41 2022-05-11 17:30:36 25 29
reaching_go_spout_cued_uncued 42 2022-05-11 17:32:14 5 0
reaching_go_spout_cued_uncued 42 2022-05-11 17:47:30 38 3
reaching_go_spout_cued_uncued 43 2022-05-11 19:24:51 34 57
reaching_go_spout_cued_uncued 44 2022-05-11 19:24:51 22 37
reaching_go_spout_cued_uncued 45 2022-05-11 19:27:52 25 55
reaching_go_spout_cued_uncued 41 2022-05-12 15:28:29 33 105
reaching_go_spout_cued_uncued 42 2022-05-12 15:33:00 44 88
reaching_go_spout_cued_uncued 43 2022-05-12 16:32:18 51 61
reaching_go_spout_cued_uncued 44 2022-05-12 16:32:18 30 40
reaching_go_spout_cued_uncued 45 2022-05-12 16:37:28 11 0
reaching_go_spout_cued_uncued 45 2022-05-12 16:54:09 5 2
reaching_go_spout_cued_uncued 41 2022-05-16 13:21:21 3 0
reaching_go_spout_

### Match with photometry, videos, and DeepLabCut files

The following Warning : 

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads...

is due to rsync function for photometry-pycontrol alignment

2m10.9s


In [7]:
# Find if there is a matching photometry file and if it can be used:
# rsync synchronization pulses matching between behaviour and photometry
from copy import deepcopy

exp_cohort.match_to_photometry_files(photometry_dir, rsync_chan, verbose=False)

# Find matching videos
exp_cohort.match_sessions_to_files(video_dir, ext='mp4')

# FInd matching DeepLabCut outputs files
exp_cohort.match_sessions_to_files(video_dir, ext='h5')

exp_cohort.save()

exp_cohort_copy = deepcopy(exp_cohort)


c:\Users\phar0732\.conda\envs\trialexp\lib\site-packages\sklearn\mixture\_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  cluster.KMeans(
c:\users\phar0732\documents\github\trialexp\trialexp\utils\rsync.py:100: RuntimeWarning: All-NaN slice encountered
  order_OK = ((np.nanmin(np.diff(cor_times_A)) > 0) and


## Visualise a session using Plotly

In [55]:

import plotly.graph_objects as go


%TODO

- markers
- drowdown to change time units
- express states by lines ... requires manual definition of each state

In [77]:
raw_symbols  = plotly.validators.scatter.marker.SymbolValidator().values
symbols = [raw_symbols[i+2] for i in range(0, len(raw_symbols), 12)]


session1 = exp_cohort.sessions[0]

fig = go.Figure()

keys = session1.times.keys()

for kind, k in enumerate(keys):

    sc = go.Line(x=session1.times[k]/1000, y=[k]
                 * len(session1.times[k]), name=k, mode='markers', marker_symbol=symbols[kind % 40]) # why % 40 ?
    fig.add_trace(sc)

fig.update_xaxes(title='Time (s)')


fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["type", "milliseconds"],
                    label="milliseconds",
                    method="restyle"
                ),
                dict(
                    args=["type", "seconds"],
                    label="seconds",
                    method="restyle"
                ),
                dict(
                    args=["type", "minutes"],
                    label="minutes",
                    method="restyle"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.04,
            xanchor="left",
            y=1.2,
            yanchor="top"
        ),
    ]
)

fig.update_layout(
    annotations=[
        dict(text="Time unit:", showarrow=False,
        x=-0, y=1.14, xref="paper", yref="paper", align="left")
    ]
)

fig.show()


In [8]:
exp_cohort.sessions[0].plot_session()


c:\Users\phar0732\.conda\envs\trialexp\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [20]:
session = exp_cohort.sessions[50]

In [58]:

symbols

['circle',
 'square',
 'diamond',
 'cross',
 'x',
 'triangle-up',
 'triangle-down',
 'triangle-left',
 'triangle-right',
 'triangle-ne',
 'triangle-se',
 'triangle-sw',
 'triangle-nw',
 'pentagon',
 'hexagon',
 'hexagon2',
 'octagon',
 'star',
 'hexagram',
 'star-triangle-up',
 'star-triangle-down',
 'star-square',
 'star-diamond',
 'diamond-tall',
 'diamond-wide',
 'hourglass',
 'circle-cross',
 'square-cross',
 'diamond-cross',
 'cross-thin',
 'asterisk',
 'hash-dot',
 'y-down',
 'y-right',
 'line-ns',
 'line-nw',
 'arrow-down',
 'arrow-right',
 'arrow-bar-down',
 'arrow-bar-right']

In [56]:
from plotly.graph_objects.scatter import Marker

ModuleNotFoundError: No module named 'plotly.graph_objects.scatter'

In [75]:
# ev_array = np.array(session.df_events.loc[(trial), event_col])
event_cols = [event_col for event_col in session.df_events.columns if '_trial_time' in event_col]
event_names = [event_col.split('_trial_time')[0] for event_col in event_cols]




print(ev_trial_nb.shape, ev_times.shape)

(672,) (672,)


In [71]:
session.df_events[event_cols[0]].apply(lambda x: np.array(x)).values

array([array([11533, 11625, 11841, 11882, 11996, 12221, 12298, 19644, 20393]),
       array([6925, 7020]), array([], dtype=float64),
       array([1217, 6169, 6933, 8663]), array([], dtype=float64),
       array([7103, 7131]), array([3552, 4484, 4550]),
       array([1526, 1602, 1842, 3831, 5719]), array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64), array([3194]),
       array([], dtype=float64), array([], dtype=float64), array([8191]),
       array([9257, 9321]), array([4273, 4331]), array([], dtype=float64),
       array([12585, 12734, 12818, 12841]), array([], dtype=float64),
       array([ 1328,  1436,  1466,  8791,  9346,  9365, 20876]),
       array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), array([9672]), array([], dtype=float64),
       array([ 5915,  6132,  6590,  6631, 33613]), array([7141]),
       array([13355, 17050, 34361]), array([7703, 7762]),
       array([21708, 21762, 22395, 25109, 25235, 51463, 522

In [153]:
plot_names =  [trig + ' ' + event for event in session.events_to_process for trig in session.triggers]
plot_names

['cued bar',
 'uncued bar',
 'cued bar_off',
 'uncued bar_off',
 'cued spout',
 'uncued spout',
 'cued hold_start',
 'uncued hold_start']

In [154]:

from plotly.validators.scatter.marker import SymbolValidator
from plotly.subplots import make_subplots

def plot_trials(session):

    raw_symbols  = SymbolValidator().values
    symbols = [raw_symbols[i+2] for i in range(0, len(raw_symbols), 12)]

    event_cols = [event_col for event_col in session.df_events.columns if '_trial_time' in event_col]
    event_names = [event_col.split('_trial_time')[0] for event_col in event_cols]

    plot_names =  [trig + ' ' + event for event in session.events_to_process for trig in session.triggers]

    fig = make_subplots(
        rows= len(event_cols), 
        cols= len(session.triggers), 
        shared_xaxes= True,
        subplot_titles= plot_names
    )

    for trig_idx, trigger in enumerate(session.triggers):
        
        # sub-selection of df_events based on trigger, should be condition for event_dataset class
        df_subset = session.df_events[session.df_events.trigger == trigger]

        for ev_idx, event_col in enumerate(event_cols):

            ev_times = df_subset[event_cols[0]].apply(lambda x: np.array(x)).values
            ev_trial_nb = [np.ones(len(array)) * df_subset.index[idx] for idx, array in enumerate(ev_times)]

            ev_trial_nb = np.concatenate(ev_trial_nb)
            ev_times =  np.concatenate(ev_times)

            fig.add_trace(
                go.Scatter(
                    x=ev_times/1000,
                    y=ev_trial_nb,
                    name=event_names[ev_idx],
                    mode='markers',
                    marker_symbol=symbols[ev_idx % 40]

                ), row= ev_idx+1, col = trig_idx+1)

            fig.update_xaxes(
                ticks="inside",
                ticklen=6,
                tickwidth=2,
                tickfont_size=12,
                range=[session.trial_window[0]/1000, session.trial_window[1]/1000],
                showline=True,
                linecolor='black'
                )
            
            fig.update_yaxes(    
                ticks="inside",
                ticklen=6,
                tickwidth=2,   
                tickfont_size=12,
                showline=True,
                linecolor='black'
                )


    fig.update_layout(

        height=800,
        width=600,
    )

    fig.show()
            

In [156]:
session.plot_trials()

In [119]:
exp_cohort.sessions[50].df_events

,timestamp,trigger,valid,success,bar_trial_time,bar_off_trial_time,spout_trial_time,hold_start_trial_time
trial_nb,,,,,,,,
1,3817,cued,True,False,"[11533, 11625, 11841, 11882, 11996, 12221, 122...","[11610, 11826, 11867, 11976, 12012, 12282, 125...",[],[11443]
2,28183,cued,True,True,"[6925, 7020]","[6886, 6941]","[856, 931]",[3504]
3,39828,cued,True,False,[],[],[],[9718]
4,52127,cued,True,True,"[1217, 6169, 6933, 8663]","[1157, 6044, 6876, 8634]","[1171, 1246]",[3903]
5,65203,cued,True,False,[],[],[],[10012]
...,...,...,...,...,...,...,...,...
102,2276775,uncued,True,False,[35846],[10355],[],[19402]
103,2315518,uncued,True,False,[],[],[],[5771]
104,2324432,uncued,True,False,[],[],[],[28488]
